In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sivarazadi/wikiart-art-movementsstyles")

print("Path to dataset files:", path)

Mounting files to /kaggle/input/wikiart-art-movementsstyles...
Path to dataset files: /kaggle/input/wikiart-art-movementsstyles


In [7]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm.notebook import tqdm


In [9]:
IMG_SIZE = 224
BATCH_SIZE = 32

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])

data_dir = "/kaggle/input/wikiart-art-movementsstyles"

train_dataset = ImageFolder(data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
num_classes = len(train_dataset.classes)

print("Classes:", train_dataset.classes)


Classes: ['Academic_Art', 'Art_Nouveau', 'Baroque', 'Expressionism', 'Japanese_Art', 'Neoclassicism', 'Primitivism', 'Realism', 'Renaissance', 'Rococo', 'Romanticism', 'Symbolism', 'Western_Medieval']


In [12]:
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, num_classes)


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)


In [ ]:
EPOCHS = 3

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss:.4f}")


  0%|          | 0/1329 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (96714256 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 1/3, Loss: 2592.9401


  0%|          | 0/1329 [00:00<?, ?it/s]

In [ ]:
torch.save(model.state_dict(), "art_style_resnet18.pth")
print("Model saved!")


In [ ]:
import PIL.Image as Image

def predict_image(path):
    img = Image.open(path).convert("RGB")
    img = transform(img).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(img)
        pred = output.argmax(dim=1).item()

    return train_dataset.classes[pred]


In [ ]:
# ---------- EXAMPLE: TEST ON ONE IMAGE ----------
# Replace path with any image from your dataset
example_path = list(train_data.filepaths)[0]
style, conf = predict_style(example_path)

print("Example prediction:")
print("Style:", style)
print("Confidence:", conf)

In [ ]:
#  GRADIO interface
import gradio as gr

def classify_image(img):
    img = img.resize((IMG_SIZE, IMG_SIZE))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)
    class_index = np.argmax(preds)
    class_name = list(train_data.class_indices.keys())[class_index]
    confidence = float(preds[0][class_index])

    return {class_name: confidence}

demo = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(),
    title="Art Style Classifier"
)

demo.launch(debug=False)